## Notebook Setup

### Imports
* **Run this cell immediately after opening the notebook by placing the cursor in the cell and hitting `SHIFT + ENTER`.**
* Otherwise, leave this cell alone

In [1]:
# Imports for telling Python where to find IRScout Python modules
import sys
import os
import re
sys.path.append('..\\Server')
    
# Set Current working directory to irsScouting2017\\Server
ptn = re.compile("jupyter$")
if ptn.search(os.getcwd()) is not None:
    os.chdir("..\\Server")
os.getcwd()

# IRScout Python Modules
import Server.scouting.output as out
from Server.scouting.event import EventDal

# Use the Bokeh plotting package
import holoviews as hv

# Other useful modules
import pandas as pd
import sqlalchemy as sa

# Get a database connection
connection_string = ('postgresql://irs1318:irs1318'
                     '@localhost:5432/scouting')
engine = sa.create_engine(connection_string)
conn = engine.connect()

ModuleNotFoundError: No module named 'Server'

### Set Defaults
### Set Event
* **Next, set `current_event` to the applicable FIRST API event code and run this cell.**

In [ ]:
# Set Chart Size
# bc.defaults.height = 300
# bc.defaults.width = 1200

# Set Current Event
current_event = "turing"
EventDal.set_current_event(current_event)
EventDal.get_current_event()

### Export Excel Ranking File
* Run this cell to export the rankings spreadsheet.

In [ ]:
fpath = out.get_Path("Report")
out.get_report(fpath)

### Get a Dataframe Containing all Measures

In [ ]:
# Get Measures Dataframe
sql_measures = sa.text(
    "SELECT events.name AS event, matches.name AS match, levels.name AS level, dates.name AS date, "
    "teams.name AS team, alliances.name AS alliance, "
    "stations.name AS station, phases.name AS phase, actors.name AS actor, tasks.name AS task, "
    "measures.capability AS capability, measures.successes as successes, measures.attempts AS attempts, "
    "measures.cycle_times as cycle_times, measuretypes.name AS measuretype, reasons.name AS reason, "
    "attempts.name AS attempt "
    "FROM ((((((((((((measures FULL OUTER JOIN teams ON measures.team_id=teams.id) "
    "LEFT JOIN events ON measures.event_id=events.id) "
    "LEFT JOIN matches ON measures.match_id=matches.id) "
    "LEFT JOIN levels ON measures.level_id=levels.id) "
    "LEFT JOIN dates ON measures.date_id=dates.id) "
    "LEFT JOIN alliances ON measures.alliance_id=alliances.id) "
    "LEFT JOIN stations ON measures.station_id=stations.id) "
    "LEFT JOIN phases ON measures.phase_id=phases.id) "
    "LEFT JOIN actors ON measures.actor_id=actors.id) "
    "LEFT JOIN tasks ON measures.task_id=tasks.id) "
    "LEFT JOIN reasons ON measures.reason_id=reasons.id) "
    "LEFT JOIN measuretypes ON measures.measuretype_id=measuretypes.id) "
    "LEFT JOIN attempts ON measures.attempt_id=attempts.id "
    "WHERE events.name='" + current_event + "';")
measures = pd.read_sql(sql_measures, conn)
measures.query("task == 'shootHighBoiler'").query("phase == 'auto' or phase == 'teleop'")
#measures.query("task == 'climbRope' and phase == 'finish'")"

### Get Number of Completed Matches"

In [ ]:
# Set recent_matches to n to filter out all but the last n matches.
recent_matches= 12

select_recent_matches = sa.text(
    "with current AS (SELECT s.event, s.match, date from schedules sched, "
    "status s WHERE sched.event = s.event "
    "AND sched.match = s.match limit 1 ) "


    "SELECT * FROM ( "
    "SELECT row_number() over (partition by team order by sched.date desc) as r, "
    " sched.* from schedules sched, current c WHERE sched.event = c.event and sched.date <= c.date )"
    " row_schedule WHERE row_schedule.r <= " + str(
        recent_matches) + " ORDER by team, date desc")
df = pd.read_sql(select_recent_matches, conn)
num_matches = df[['team', 'level']].groupby('team').count()
num_matches.columns = ['matches']

### Create and Save Gears Chart
* This cell creates the gears chart and saves it to Server/web/data as a static, interactive html file.

**TODO:** Add an event code and timestamp to gears chart file name.

In [ ]:
# Create summary data frame for placing gears
gears_df = measures.query("task == 'placeGear'").query("phase == 'auto' or phase == 'teleop'").groupby('team').sum()
gears_df['average'] = gears_df['successes'] / num_matches['matches']
gears_df = gears_df.sort_values('average', ascending = False)

gears_df

# Create gears chart in Bokeh
# gears_chart = bc.Bar(gears_df, values = 'average', label = bca.cat(sort = False), legend = None, color = 'purple',
#                      title = 'Average Gears Placed per Match', ylabel = "Gears Placed")
# bc.output_file('web/data/gears_chart.html', mode = 'inline')
# gears_df = measures.query("task == 'climbRope'").query("phase == 'finish'").groupby('team').sum()
# gears_df = gears_df.sort_values('successes', ascending = False)

# # Create gears chart in Bokeh
# gears_chart = bc.Bar(gears_df, values = 'successes', label = bca.cat(sort = False), legend = None, color = 'purple',
#                     title = 'Total Gears Placed', ylabel = "Gears Placed")
# bc.output_file('web/data/gears_chart.html')
# bc.show(gears_chart)

In [ ]:
# Create summary data frame for climbing rope
climbs_df = measures.query("task == 'climbRope'").query("phase == 'finish'").groupby('team').count()
climbs_df['average'] = climbs_df['successes'] / num_matches['matches']
climbs_df = climbs_df.sort_values('average', ascending = False)

# Create gears chart in Bokeh
climb_chart = bc.Bar(climbs_df, values = 'average', label = bca.cat(sort = False), legend = None, color = 'DarkGoldenRod',
                   title = 'Total Times Climbed', ylabel = "Times Climbed")
bc.output_file('web/data/climb_chart.html', mode = 'inline')
bc.show(climb_chart)